# Raster Preprocessing in Python
This notebook will demonstrate how to
1. Resample and reproject raster
2. Mask rasters
3. Reclassify raster values

In [2]:
# IMPORT LIBRARIES####
import rasterio as rio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.enums import Resampling

import numpy as np

import pandas as pd

import geopandas as gpd
import json
import os

In [2]:
os.chdir(r"/Volumes/volume 1/GIS Projects/Research/240105/researchProj2/exported")

## Reproject

In [4]:
def reprojectRaster(inPath, 
                    outPath,
                    outCrs):
   with rio.open(inPath) as src:
       # Calculate Output Meta Data====
       outTrans, outW, outH = rio.warp.calculate_default_transform(
           src.crs, outCrs, src.width, src.height, *src.bounds
       )

       # Update Output Meta Data====
       profile = src.profile.copy()
       profile.update({
           'crs': outCrs,
           'transform': outTrans,
           'width': outW,
           'height': outH
       })

       # Write Data with Reprojection====
       with rio.open(outPath, 'w', **profile) as dst:
           rio.warp.reproject(
               source = rio.band(src, 1),
               destination = rio.band(dst, 1),
               src_transform = src.transform,
               src_crs = src.crs,
               dst_transform = outTrans,
               dst_crs = outCrs,
               resampling=Resampling.nearest  # You can choose a different resampling method if needed
           )

In [5]:
# APPLY FUNCTION####
d = rio.open("lulc2005OriginalRaster.tif")
crs = d.crs

reprojectRaster("westMALu2016.tif",
                "lulc2016Reproj.tif",
                crs)

## Resample

In [17]:
# DEFINE FUNCTION FOR REPROJECTION####
def resampleRaster(modifiedFile,
                   targetFile,
                   outPath):
    with rio.open(modifiedFile) as src:
        with rio.open(targetFile) as dst:
        # Define Transformation Parameters====
            outTrans, outWidth, outHeight = calculate_default_transform(
                src.crs,
                dst.crs,
                dst.width,
                dst.height,
                *dst.bounds
            )
        
        # Get Meta Information and OverWrite====
        outMeta = src.meta.copy()
        outMeta.update({
            "crs": dst.crs,
            "transform": outTrans,
            "width": outWidth,
            "height": outHeight,
            "nodata": 0
        })

        # Write Data with Assigning Meta====
        with rio.open(outPath,
                      "w",
                      **outMeta) as r:
            # Write file by reprojecting every band----
            for i in range(1, src.count + 1):
                reproject(
                    source = rio.band(src, i),
                    destination = rio.band(r, i),
                    src_transform = src.transform,
                    src_crs = src.crs,
                    dst_transform = outTrans,
                    dst_crs = dst.crs,
                    resampling = Resampling.nearest
                )

In [18]:
# APPLY FUNCTION####
resampleRaster("lulc2016Reproj.tif",
               "lulc2005OriginalRaster.tif",
               "lulc2016Resampled.tif")

In [8]:
# CHECK OUTPUT####
ref = rio.open("lulc2005OriginalRaster.tif")
modified = rio.open("lulc2016Resampled.tif")
print(np.unique(modified.read(1)))

print("Match CRS?: ", ref.crs == modified.crs)

print("Same Shape?: ", np.shape(ref.read(1)) == np.shape(modified.read(1)))

[ 0  2  5  6  7  8  9 10 12 13 14 15 19 20 21 22]
Match CRS?:  True
Same Shape?:  True


## Mask Raster

In [9]:
# CONVERT SHAPE FILE TO JSON FORMAT####
def changeShpToJson(pathToBoundary,
                    pathToRaster):
    g = gpd.read_file(pathToBoundary).dissolve()
    r = rio.open(pathToRaster)

    # Align CRS====
    print(g.crs)
    print(r.crs)
    
    if g.crs != r.crs:
        g = g.to_crs(r.crs)

    # Convert Geopandas Geometry to Json====
    return [json.loads(g.to_json())["features"][0]["geometry"]]

In [10]:
# APPLY FUNCTION####
coords = changeShpToJson("westCounties.shp",
                         "lulc2005OriginalRaster.tif")
print(coords)

EPSG:26986
EPSG:26986
[{'type': 'Polygon', 'coordinates': [[[142138.3599999994, 864746.5], [141614.70000000298, 864757.5599999987], [137933.45000000298, 864835.1900000013], [136188.88000000268, 864871.3099999987], [128209.39999999851, 865065.879999999], [127585.25999999791, 865075.5599999987], [125687.43999999762, 865132.8099999987], [120644.75, 865315.5], [117237.38000000268, 865359.5599999987], [116432.88000000268, 865371.9400000013], [114580.03000000119, 865416.0599999987], [113093.64999999851, 865225.379999999], [111167.34000000358, 864982.8099999987], [110385.0700000003, 864383.0599999987], [108335.3900000006, 864458.5], [108252.10000000149, 865104.379999999], [107917.09000000358, 865114.379999999], [107525.74000000209, 865139.129999999], [106881.3900000006, 865187.0599999987], [105871.81000000238, 865243.879999999], [105594.68999999762, 865264.129999999], [100735.1799999997, 865872.629999999], [99559.89999999851, 865886.629999999], [99447.60000000149, 865889.9400000013], [99446.3

In [11]:
# MASK RASTER BY FEATURE####
def maskRaster(pathToRaster,
               boundary,
               outPath):
    r = rio.open(pathToRaster)
    import rasterio.mask as msk

    # Mask Raster====
    outImg, outTrans = msk.mask(r,
                                shapes = boundary,
                                crop = True)
    
    # Updating Meta====
    outMeta = r.meta.copy()
    outMeta.update({
        "dirver": "GTiff",
        "height": outImg.shape[1],
        "width": outImg.shape[2],
        "transform": outTrans,
        "nodata": -1,
        "dtype": np.int8,
        "crs": r.crs
    })

    # Write File====
    with rio.open(outPath, 
                  "w",
                  **outMeta) as outfile:
        outfile.write(outImg)


In [12]:
maskRaster("lulc2005OriginalRaster.tif",
           coords,
            "lulc2005Mask.tif")
maskRaster("lulc2016Resampled.tif",
           coords,
           "lulc2016Mask.tif")

## Reclassify Raster (Binary + Out of Boundary)
This code assigns 2 to foreground pixel.

In [4]:
def reclassifyRaster(inPath,
                    foregroundClassList,
                    outPath,
                    outBoundaryOriginalValue = 0,
                    ):
    
    # Read and Convert into Array====
    r = rio.open(inPath)
    arr = r.read(1)

    # Reclassify Raster Value====
    outArr = arr.copy()

    outArr[(np.isin(arr, foregroundClassList))] = 2
    outArr[~np.isin(arr, foregroundClassList)] = 1
    
    # Use below if multiple classes----
    # quadroArr[(r1ArrUpd + r2ArrUpd == 2)] = 1
    # quadroArr[(r1ArrUpd + r2ArrUpd == 0)] = 3
    # quadroArr[(r1ArrUpd - r2ArrUpd == 1)] = 4
    # quadroArr[(r2ArrUpd - r1ArrUpd == 1)] = 2
    # quadroArr[np.isnan(r1Arr)] = 0
    
    # Overwrite 2 with 0 If out of Boundary====
    outArr[arr == outBoundaryOriginalValue] = 0

    # Write Raster with Updating Meta=====
    outMeta = r.meta.copy()
    with rio.open(outPath,
                  "w",
                  width= r.width,
                  height = r.height,
                  count = 1,
                  crs = r.crs,
                  transform = r.transform,
                  dtype = "uint8",
                  ) as r:
        r.write(outArr, 1)

In [19]:
# APPLY FUNCTION####
reclassifyRaster("lulc2005Mask.tif",
                 [3, 37],
                 "lulc2005Forest2.tif")
reclassifyRaster("lulc2016Mask.tif",
                 [9, 10],
                 "lulc2016Forest2.tif")

## Reclassification (Multiple Classes)

In [5]:
f = r"fod4conn/lulc2016Forest2_fos-fad_5class_163/lulc2016Forest2_fos-fad_5class_163.tif"
outPath = r"fad2016_4conn.tif"
r = rio.open(f)
arr = r.read(1)

print(np.unique(arr))


[ 12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29
  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 101 102]


In [6]:
outArr = arr.copy()

outArr[(arr > 0) & (arr < 60)] = 1
outArr[(arr >= 60) & (arr < 101)] = 2
outArr[(arr == 101)] = 3
outArr[(arr == 102)] = 0

outMeta = r.meta.copy()
with rio.open(outPath,
              "w",
              width= r.width,
              height = r.height,
              count = 1,
              crs = r.crs,
              transform = r.transform,
              dtype = "uint8",
              ) as r:
        r.write(outArr, 1)


### Another way: use of a look up table

In [11]:
def reclassifyRasterFromDict(pathToRaster,
                             lookUpTable,
                             outPath):
    r = rio.open(pathToRaster)
    arr = r.read(1)
    outArr = arr.copy()

    for row in lookUpTable:
        print(row)
        original = row[0]
        print(original)

        new = row[1]
        print(new)

        outArr[arr == original] = new

    outMeta = r.meta.copy()
    with rio.open(outPath,
                  "w",
                  **outMeta) as writer:
        writer.write(outArr, 1)

In [16]:
os.chdir(r"/Applications/GuidosToolbox/data/ma")
t = [(35, 33),
     (67, 65),
     (101, 1),
     (103, 3),
     (109, 9),
     (117, 17),
     (129, 225),
     (133, 33),
     (135, 33),
     (137, 33),
     (165, 65),
     (167, 65),
     (169, 65)
     ]

reclassifyRasterFromDict("mspa2005_4111.tif",
                         t,
                         "mspa2005_4111Formatted.tif")
reclassifyRasterFromDict("mspa2016_4111.tif",
                         t,
                         "mspa2016_4111Formatted.tif")

(35, 33)
35
33
(67, 65)
67
65
(101, 1)
101
1
(103, 3)
103
3
(109, 9)
109
9
(117, 17)
117
17
(129, 225)
129
225
(133, 33)
133
33
(135, 33)
135
33
(137, 33)
137
33
(165, 65)
165
65
(167, 65)
167
65
(169, 65)
169
65

(35, 33)
35
33
(67, 65)
67
65
(101, 1)
101
1
(103, 3)
103
3
(109, 9)
109
9
(117, 17)
117
17
(129, 225)
129
225
(133, 33)
133
33
(135, 33)
135
33
(137, 33)
137
33
(165, 65)
165
65
(167, 65)
167
65
(169, 65)
169
65



In [15]:
# CHECK RESULT####
arro = rio.open("mspa2016_4111.tif").read(1)
arrn = rio.open("mspa2016_4111Formatted.tif").read(1)

# Print Unique Values====
print("Unique: ", np.unique(arro))
print(np.unique(arrn))

# Check If the Size is the Same for Some of the Category====
print("Size of Several Categories: ", np.sum((arro == 101) | (arro == 1)) == np.sum(arrn == 1))
np.sum((arro == 33) | (arro == 35) | (arro == 133) | (arro == 135) | (arro == 137)) == np.sum(arrn == 33)


Unique:  [  0   1   3   9  17  33  35  65  67 100 101 103 105 109 117 129 133 135
 137 165 167 169 220]
[  0   1   3   7   9  17  33  65 100 105 220 225]
Size of Several Categories:  True


True